# Compare Datasets

This code takes in two files (csv) and then compares the selected data from each to determine which is greater. The code then sets the appropriate LED to high.

Example based on the following pin setup:


| Data | 1st GPIO Pin | 2nd GPIO Pin |
|---|---|---|
| oxidising | 17->GND | 22 ->GND |
| reducing | 18->GND | 21 -> GND |
| nh3 | 19->GND | 20->GND |

In [2]:
print ("Current working directory:")
!pwd
print ("\nAvailable files:")
!ls *.csv

Current working directory:
/home/pi/Desktop/data

Available files:
recordings-072921-161554.csv  recordings-073021-114357.csv
recordings-073021-111006.csv  recordings-073021-151701.csv


In [3]:
# Insert the file names
datafile_1 = 'recordings-073021-151701.csv'
datafile_2 = 'recordings-073021-114357.csv'

# Libraries needed to do the work
import pandas as pd # The Pandas library makes it easier to work with data
import numpy as np # Numpy adds additional numerical methods (math functions)
import matplotlib.pyplot as plt
from matplotlib import style

style.use('fivethirtyeight')

# Read in the csv datafile from above, and save it to the Pandas "dataframe" named sensor_data
# and add the header names to match the data
sensor_data1 = pd.read_csv(datafile_1)
sensor_data2 = pd.read_csv(datafile_2)

# Print the first 5 lines of the dataframe named sensor_data
print(sensor_data1.head())
print(sensor_data2.head())


       date      time  temperature   humidity    pressure       lux  \
0  07/30/21  15:17:01    22.501424  94.338431  695.425250  29.04625   
1  07/30/21  15:17:02    32.140563  42.397307  989.865529  12.96480   
2  07/30/21  15:17:03    32.141544  42.397305  989.867037  12.96480   
3  07/30/21  15:17:04    32.144486  42.397298  989.871561  12.96480   
4  07/30/21  15:17:05    32.146121  42.403372  989.869022  12.96480   

      oxidizing       reducing          nh3  
0  15544.715447  695219.512195  76473.11828  
1  15544.715447  695219.512195  76473.11828  
2  15544.715447  704493.827160  76473.11828  
3  15544.715447  695219.512195  76473.11828  
4  15544.715447  695219.512195  76473.11828  
       date      time  temperature   humidity    pressure        lux  \
0  07/30/21  11:43:57    22.501424  94.338431  695.425250  614.49255   
1  07/30/21  11:43:58    28.216535  55.997998  991.649235  793.40635   
2  07/30/21  11:44:00    28.184173  56.321054  991.644359  797.13150   
3  07/30/

# Setup the GPIO pins on the RPi

In [4]:
import RPi.GPIO as GPIO   ## Library that allows the GPIO pins to be used
import time
##Setup the GPIO Pins
for pin in range(17,23):
    GPIO.setwarnings(False)  ## Removes any warnings if the pin was previously used
    GPIO.setmode(GPIO.BCM)   ## Allows the use of the GPIO pin numbers instead of the physical pin numbers
    GPIO.setup(pin,GPIO.OUT)
    GPIO.output(pin,GPIO.LOW) ## Sets GPIO pin output voltage low/off

# Configure wanted comparison data to the GPIO pins

* For the data column change the **"data_to_check"** list variable to the ones wanted

* For the two positive GPIO pins associated with the two LEDs for that data column change the **"pins_to_use"** nested list variable

**Example:**
```
data_to_check = ["oxidizing","reducing","nh3"]

pins_to_use = [[17,22],[18,21],[19,20]] 
```

_Output:_

Using the following GPIO pins for the given data:
oxidizing
[17, 22]
reducing
[18, 21]
nh3
[19, 20]

In [7]:
# Insert the data header titles you want to compare between files
data_to_check = ["oxidizing","reducing","nh3"]

# Insert the pins you want to use for each data set
pins_to_use = [[17,22],[18,21],[19,20]]

# Prints out pin configuration for the data headers
print("Using the following GPIO pins for the given data:")
for i in range(len(data_to_check)):   
    print(data_to_check[i])
    print(pins_to_use[i])
    


Using the following GPIO pins for the given data:
oxidizing
[17, 22]
reducing
[18, 21]
nh3
[19, 20]


# Run through the two datasets and turn on the pin for each that is greater

In [37]:
# First Ensures all LEDS are off
for pin in range(17,23):
    GPIO.output(pin,GPIO.LOW)

data_pins =[]
for i in range(len(data_to_check)):
    compare_data = pd.DataFrame()
    # make a pandas dataframe object with the data "i" from the two files
    data = [sensor_data1[data_to_check[i]],sensor_data2[data_to_check[i]]]
    compare_data = pd.concat(data, axis=1)
    #show the first 5 lines of the new dataframe
    print(compare_data.head())
    # run through the dataframe to compare the two values
    pin_list = []
    for x in range(len(compare_data)):
        #Check each row and set the pins
        if compare_data.iloc[i,0] >= compare_data.iloc[i,1]:
            pin_list.append(pins_to_use[i][0])
        else:
            pin_list.append(pins_to_use[i][1])
    data_pins.append(pin_list)
    print(data_to_check[i],data_pins)

# Now to run through the LED sequence turning the LEDS on then off
for leds in range(len(data_pins[0])):
    for data_sensortype in range(len(data_to_check)):
        GPIO.output(data_pins[data_sensortype][leds],GPIO.HIGH)
        print("Setting HIGH pin:",str(data_pins[data_sensortype][leds]))
    time.sleep(1)
    for data_sensortype in range(len(data_to_check)):
        GPIO.output(data_pins[data_sensortype][leds],GPIO.LOW)
        print("Setting LOW pin:",str(data_pins[data_sensortype][leds]))
    

      oxidizing     oxidizing
0  15544.715447  16813.238771
1  15544.715447  16899.408284
2  15544.715447  16985.781991
3  15544.715447  16985.781991
4  15544.715447  17072.360617
oxidizing [[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22]]
        reducing       reducing
0  695219.512195  891692.307692
1  695219.512195  877333.333333
2  704493.827160  877333.333333
3  695219.512195  863402.985075
4  695219.512195  877333.333333
reducing [[22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22], [21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21

Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: 22
Setting LOW pin: 21
Setting LOW pin: 20
Setting HIGH pin: 22
Setting HIGH pin: 21
Setting HIGH pin: 20
Setting LOW pin: